This script starts batch job which reads data from Apartment  maintenance API.

In [0]:
# imports
from pyspark.sql.types import StringType, IntegerType, TimestampType, DateType, DoubleType, StructType, StructField
import schedule
import time
import requests
import datetime
import pandas as pd
from pyspark.sql.functions import lit
 


In [0]:
# Schema for Apartment Maintenance
apt_maintenance_schema = StructType([
            StructField("Maintenance_id", IntegerType(), False),
            StructField("Apartment_number", IntegerType(), True),
            StructField("Mdate", StringType(), False),
            StructField("Issue_reported", StringType(), True),
            StructField("Contractor_id", IntegerType(), True), 
            StructField("Resolution", StringType(), True), 
            StructField("Status", StringType(), True),
            StructField("Charges_incurred", StringType(), True),
            StructField("event_time", StringType(), True)
])

In [0]:
apt_maintenance_path = "/FileStore/users/apt_maintenance/inprogress" 
dbutils.fs.mkdirs(apt_maintenance_path)
apartment_maintenance_url = "https://my.api.mockaroo.com/apartment_maintenance.json?key=6af9c3e0"



In [0]:
# Save csv file from api url
# '/FileStore/apartment/inprogress/'
def getCSV_FromUrl(url, schema, path_to_save):  
  df = spark.createDataFrame(pd.read_csv(url), schema)
  print(df.count())
  ts = time.time()
  st = datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d_%H_%M')
  print(st)
  df_with_batch = df.withColumn("fetch_time", lit(datetime.datetime.fromtimestamp(ts).strftime('%Y_%m_%d_%H_%M_%S')))
  fileName = path_to_save + st + '.tmp'
  fileprefix = path_to_save
  df_with_batch.coalesce(1).write.format("com.databricks.spark.csv") \
    .option("header", True) \
    .option("quote", "") \
    .option("multiLine", True) \
    .save(fileName)  #saved to the FileStore
    
  fileList =  dbutils.fs.ls(fileName)

  csvFileLocation = ''
  for fileInfo in fileList:   
    if ".csv" in fileInfo.path:
      print("this file is csv file.." )
      print(fileInfo.path)
      csvFileLocation = fileprefix + fileInfo.name
      
      dbutils.fs.cp(fileInfo.path,fileprefix)
      dbutils.fs.rm(fileName,recurse=True)

In [0]:

def job():
  print("calling CSV load function")
  getCSV_FromUrl(apartment_maintenance_url, apt_maintenance_schema, apt_maintenance_path)

In [0]:
schedule.every(10).seconds.do(job)
 

while True:
    schedule.run_pending()
    time.sleep(1)

calling CSV load function

--------------------------------------------------------------------------- 
 HTTPError Traceback (most recent call last)
 <command-3248458478020621> in <module> 
 3 
 4 while True : 
 ----> 5 schedule . run_pending ( ) 
 6 time . sleep ( 1 ) 

 /databricks/python/lib/python3.5/site-packages/schedule/__init__.py in run_pending () 
 561 : data : ` default scheduler instance < default_scheduler > ` . 
 562 """
 --> 563 default_scheduler . run_pending ( ) 
 564 
 565 

 /databricks/python/lib/python3.5/site-packages/schedule/__init__.py in run_pending (self) 
 92 runnable_jobs = ( job for job in self . jobs if job . should_run ) 
 93 for job in sorted ( runnable_jobs ) : 
 ---> 94 self . _run_job ( job ) 
 95 
 96 def run_all ( self , delay_seconds = 0 ) : 

 /databricks/python/lib/python3.5/site-packages/schedule/__init__.py in _run_job (self, job) 
 145 
 146 def _run_job ( self , job ) : 
 --> 147 ret = job . run ( ) 
 148 if isinstance ( ret , CancelJob ) or ret is CancelJob : 
 149 self . cancel_job ( job ) 

 /databricks/python/lib/python3.5/site-packages/schedule/__init__.py in run (self) 
 464 """
 465 logger . info ( 'Running job %s' , self ) 
 --> 466 ret = self . job_func ( ) 
 467 self . last_run = datetime . datetime . now ( ) 
 468 self . _schedule_next_run ( ) 

 <command-3248458478020618> in job () 
 2 def job ( ) : 
 3 print ( "calling CSV load function" ) 
 ----> 4 getCSV_FromUrl ( apartment_maintenance_url , apt_maintenance_schema , apt_maintenance_path ) 

 <command-3248458478020614> in getCSV_FromUrl (url, schema, path_to_save) 
 2 # '/FileStore/apartment/inprogress/' 
 3 def getCSV_FromUrl ( url , schema , path_to_save ) : 
 ----> 4 df = spark . createDataFrame ( pd . read_csv ( url ) , schema ) 
 5 print ( df . count ( ) ) 
 6 ts = time . time ( ) 

 /databricks/python/lib/python3.5/site-packages/pandas/io/parsers.py in parser_f (filepath_or_buffer, sep, delimiter, header, names, index_col, usecols, squeeze, prefix, mangle_dupe_cols, dtype, engine, converters, true_values, false_values, skipinitialspace, skiprows, skipfooter, nrows, na_values, keep_default_na, na_filter, verbose, skip_blank_lines, parse_dates, infer_datetime_format, keep_date_col, date_parser, dayfirst, iterator, chunksize, compression, thousands, decimal, lineterminator, quotechar, quoting, doublequote, escapechar, comment, encoding, dialect, tupleize_cols, error_bad_lines, warn_bad_lines, delim_whitespace, low_memory, memory_map, float_precision) 
 700 skip_blank_lines=skip_blank_lines)
 701 
 --> 702 return _read ( filepath_or_buffer , kwds ) 
 703 
 704 parser_f . __name__ = name

 /databricks/python/lib/python3.5/site-packages/pandas/io/parsers.py in _read (filepath_or_buffer, kwds) 
 411 compression = _infer_compression ( filepath_or_buffer , compression ) 
 412 filepath_or_buffer, _, compression, should_close = get_filepath_or_buffer(
 --> 413 filepath_or_buffer, encoding, compression)
 414 kwds [ 'compression' ] = compression
 415 

 /databricks/python/lib/python3.5/site-packages/pandas/io/common.py in get_filepath_or_buffer (filepath_or_buffer, encoding, compression, mode) 
 200 
 201 if _is_url ( filepath_or_buffer ) : 
 --> 202 req = _urlopen ( filepath_or_buffer ) 
 203 content_encoding = req . headers . get ( 'Content-Encoding' , None ) 
 204 if content_encoding == 'gzip' : 

 /usr/lib/python3.5/urllib/request.py in urlopen (url, data, timeout, cafile, capath, cadefault, context) 
 161 else : 
 162 opener = _opener
 --> 163 return opener . open ( url , data , timeout ) 
 164 
 165 def install_opener ( opener ) : 

 /usr/lib/python3.5/urllib/request.py in open (self, fullurl, data, timeout) 
 470 for processor in self . process_response . get ( protocol , [ ] ) : 
 471 meth = getattr ( processor , meth_name ) 
 --> 472 response = meth ( req , response ) 
 473 
 474 return response

 /usr/lib/python3.5/urllib/request.py in http_response (self, request, response) 
 580 if not ( 200 <= code < 300 ) : 
 581 response = self.parent.er